In [1]:
from math import sin, cos, sqrt, atan2, radians
from urllib.parse import quote
import json
import pandas as pd
import requests
import folium
#different modules:
#math: calculate distance
#urllib.parse,quote: required by the api provider. To combine url components
#json,pandas,folium: to create table and street map
#requests: to access api
API_KEY= 'gxy7IYotnmfdd1SrvXfy1hfG5z4HfU0pGbbVPclKWAy07uqNviLGHrnzp0J_Nt1hJ_vQd4AnIX3LQzh1SV9vybXQpR_MpvhhZTIUrx8b6K0Qp4yfU0wrPDQE_wDXXXYx'
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
#different constants required to call the api

try:

    def request(host, path, api_key, url_params=None):
        url_params = url_params or {}
        url = '{0}{1}'.format(host, quote(path.encode('utf8')))
        headers = {'Authorization': 'Bearer %s' % api_key,}
        response = requests.request('GET', url, headers=headers, params=url_params)

        return response.json()
    #this function calls the api service and return a json file

    def search(api_key, term, location):

        url_params = {
            'term': term.replace(' ', '+'),
            'location': location.replace(' ', '+'),
            'limit': SEARCH_LIMIT
        }
        return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)
#this funciton formats the parameters and call the api using request function
#two functions are modified from yelp's example at:'https://github.com/Yelp/yelp-fusion/tree/master/fusion/python'

    def get_coordinates(search):
        url = 'https://nominatim.openstreetmap.org/search' 
        options = { 'q' : search, 'format' : 'json'}
        response = requests.get(url, params = options)            
        geodata = response.json()
        coords = { 'lat' : float(geodata[0]['lat']), 'lng' : float(geodata[0]['lon']) }
        return coords['lat'],coords['lng']
#this funciton calls an api to get the coordinates
except KeyError:
    print('Key Error, Please try again.')
except IndexError:
    print('Index Error, Please try again.')
except HTTPError:
    print('HTTP Error, Please try again.')
#except any unknown error, print try again

In [2]:
while True:
#the main loop
    name=[]
    address=[]
    rating=[]
    location=[]
    distance=[]
    #empty lists
    R = 6373.0
    #radius of Earth, use to convert coordinates into distances
    
    get_type=input("Enter the type or the name of the restaurant(Asian,French,Japanese...etc.): ")
    #get restaurant type or name and store as a variable
    get_location=input("Enter the name of the city: ")
    SEARCH_LIMIT = input("Enter the amount of restaurants you want to show on the map: ")
    #gives this amount of results in the table and map
    restaurants=search(API_KEY,get_type,get_location)
    #get the json file and store it. call the api using user's preferences
    show_table = input("Do you want to show the table? (Y/N)").upper()
    #finds out either user wants to see the table or not
    get_distance=input("Enter your location(as detail as possible): ")
    #get user's current location
    user_location=get_coordinates(get_distance)
    #translate user's location into coordinates and stores it 

    try:
        for num in restaurants['businesses']:
            name.append(num['name'])
            address.append(num['location']['address1'])
            rating.append(num['rating'])
            location.append((num['coordinates']['latitude'],num['coordinates']['longitude']))
            #from the json file that was returned from yelp, add results to different empty lists
   
        lon1=radians(abs(user_location[1]))
        lat1=radians(abs(user_location[0]))
        for l in location:
            lon2=radians(abs(l[1]))
            lat2=radians(abs(l[0]))
            dlon = lon2 - lon1
            dlat = lat2 - lat1
            a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
            c = 2 * atan2(sqrt(a), sqrt(1 - a))
            distance.append('%.2f km'%(R * c))
            #formula to calculate distance from different coordinates
            #modified from:https://stackoverflow.com/questions/19412462/getting-distance-between-two-points-based-on-latitude-longitude
            
        series_dict = { 'Name':  name,'Address': address,'Rating on Yelp': rating,'Distance':distance}  
        table=pd.DataFrame(series_dict)
        #get different lists into a dictionary and create a table

        if show_table=='Y':
            display(table)
            #if user wants to see the table, show the table

        coords = get_coordinates(get_location)
        #get coordinate from the city that the user wants to find restaurants with
        show_map = folium.Map(location=coords, zoom_start=13)
        #create a map and place the center of the map with that city, zoom 13 times(able to show the whole city)
        dictionary={'name':name,'rating':rating,'address':address,'location':location,'user_location': user_location,'distance':distance}
        #set a new dictionary for the map
        for item in range(int(SEARCH_LIMIT)):
            loc=(dictionary['location'][item])
            message=f"Restaurant name:{dictionary['name'][item]} Rating:{dictionary['rating'][item]} Address:{dictionary['address'][item]} Distance:{dictionary['distance'][item]}"
            marker = folium.Marker(location=loc,popup=message)
            show_map.add_child(marker)
            #start a loop to add markers and messages on to the map in the range of the items in the lists
        marker=folium.Circle(radius=50,fill_color='red',fill_opacity=0.7,location=user_location,popup='User location')
        show_map.add_child(marker)
        #add a marker to show user's location
        display(show_map)
        #display the map

        test_continue=input('Do you want to find another restaurant? Press "quit" to stop the program. ')
        if test_continue == 'quit':
            break 
            #end the main loop or continue
    except KeyError or IndexError:
        print("Please enter something")
        break
    except HTTPError:
        print("HTTP problem, please try again")
        break
        #if there's some error, break the loop

Enter the type or the name of the restaurant(Asian,French,Japanese...etc.): italian
Enter the name of the city: new york
Enter the amount of restaurants you want to show on the map: 17
Do you want to show the table? (Y/N)y
Enter your location(as detail as possible): hinds hall


,Name,Address,Rating on Yelp,Distance
0,Lillo Cucina Italiana,331 Henry St,5.0,315.48 km
1,Olio e Piú,3 Greenwich Ave,4.0,311.28 km
2,Solaro,13 Carmine St,5.0,311.46 km
3,L'Artusi,228 W 10th St,4.5,311.04 km
4,Nostro East Village,75 2nd Ave,5.0,312.43 km
5,Rubirosa,235 Mulberry St,4.5,312.47 km
6,Gnoccheria Wall Street,100 Broad St,4.5,313.52 km
7,Supper,156 E 2nd St,4.0,312.97 km
8,Gaia Italian Café,251 E Houston St,4.0,313.08 km
9,Pepe Rosso Social,173 Mott St,4.0,312.72 km


Do you want to find another restaurant? Press "quit" to stop the program. yes
Enter the type or the name of the restaurant(Asian,French,Japanese...etc.): chinese
Enter the name of the city: syracuse
Enter the amount of restaurants you want to show on the map: 10
Do you want to show the table? (Y/N)y
Enter your location(as detail as possible): 505 walnut ave, syracuse,


,Name,Address,Rating on Yelp,Distance
0,Bamboo House Asian Restaurant,252 W Genesee St,4.5,1.98 km
1,Tang Flavor,413 S Warren St,4.0,1.54 km
2,Red Chili,2740 Erie Blvd E,4.0,2.98 km
3,Yang Di Chun BBQ,2042 Erie Blvd E,4.0,1.61 km
4,Old Chengdu Cafe,1113 E Fayette St,5.0,0.27 km
5,Asian Wok,4114 W Genesee St,4.0,9.09 km
6,China Pavilion,2318 W Genesee St,4.0,5.67 km
7,21 Cafe,124 Headson Dr,4.0,3.90 km
8,Flaming Grill & Buffet,244 Northern Lights Plz,3.5,7.28 km
9,Big Bang BBQ,449 N Salina St,4.5,2.03 km


Do you want to find another restaurant? Press "quit" to stop the program. quit


In [ ]:
#https://www.youtube.com/watch?v=PMjvSO3q6MI&t=4s